In [57]:
!jt -t oceans16 -T -N
#!jt -l

In [58]:
nEvents = 4e3
nThread = 5
Ecm= 27.0

ids = [111, 443, 411, -411]
N_part = len(ids)

nPSFs = ['nNNPDF30_nlo_as_0118_A2_Z1', 'nNNPDF10_nlo_as_0118_D2', 'TUJU21_nlo_2_1', 'nCTEQ15HIX_FullNuc_2_1'] #0-3
         #'nNNPDF30_nlo_as_0118_A2_Z1', 'nNNPDF20_nlo_as_0118_D2', 'nNNPDF10_nlo_as_0118_Au197', #4-6
         ##'EPPS16nlo_CT14nlo_Al27 ', 'EPPS16nlo_CT14nlo_Au197 ', '0 '] #7-9
opts = [1, 7]

In [73]:
import sys
cfg = open("/home/yoren/bnl/pythia8308/examples/Makefile.inc")
lib = "/home/yoren/bnl/pythia8308/lib"
for line in cfg:
    if line.startswith("PREFIX_LIB="): lib = line[11:-1]; break
sys.path.insert(0, lib)

options = ['SoftQCD:all=on', 'SoftQCD:inelastic=on', 'HardQCD:allwdi=on', 'HardQCD:all=on', #0-3
           'HardQCD:onlydi=on', 'PromptPhoton:all=on', 'PhaseSpace:pTHatMin=0.', 'PDF:pSet=LHAPDF6:cteq6l1',#4-7
           'MultipartonInteractions:Kfactor=0.5']

pdf_name = ['nNNPDF30', 'nNNPDF10', 'TUJU21', 'nCTEQ15',
            'nNNPDF30_nlo_as_0118_A2_Z1', 'nNNPDF20_nlo_as_0118_D2', 'nNNPDF10_nlo_as_0118_Au197',
            'EPPS16nlo_CT14nlo_Al27', 'EPPS16nlo_CT14nlo_Au197', '']

#!source /home/yoren/bnl/pythia8308/root_v6.26.10.Linux-ubuntu20-x86_64-gcc9.4.tar.gz/root/bin/thisroot.sh
! which root
import ROOT
#ROOT.gROOT.SetBatch()
#ROOT.ROOT.EnableThreadSafety()
#nthereds = 5
#fileName = "NICA/test.root"
#merger = ROOT.ROOT.TBufferMerger(fileName)

/home/yoren/bnl/pythia8308/root6/bin/root


In [60]:
import lhapdf
lha_npdf_vec=[]
lha_npdf_set_vec=[]

for nPDF in nPSFs:
        lha_npdf_vec.append(lhapdf.mkPDFs(nPDF))
        lha_npdf_set_vec.append(lhapdf.getPDFSet(nPDF))
        
def_npdf = lhapdf.mkPDFs("cteq6l1")

LHAPDF 6.5.3 loading all 201 PDFs in set nNNPDF30_nlo_as_0118_A2_Z1
nNNPDF30_nlo_as_0118_A2_Z1, version 1; 201 PDF members
LHAPDF 6.5.3 loading all 251 PDFs in set nNNPDF10_nlo_as_0118_D2
nNNPDF10_nlo_as_0118_D2, version 1; 251 PDF members
LHAPDF 6.5.3 loading all 59 PDFs in set TUJU21_nlo_2_1
TUJU21_nlo_2_1, version 1; 59 PDF members
LHAPDF 6.5.3 loading all 43 PDFs in set nCTEQ15HIX_FullNuc_2_1
nCTEQ15HIX_FullNuc_2_1, version 2; 43 PDF members
LHAPDF 6.5.3 loading all 1 PDFs in set cteq6l1
cteq6l1, version 4; 1 PDF members


In [61]:
def weights_PDF(result_w, lha_npdf, lha_npdf_set, pdfid, pdfx, pdfscale, pdfpdf):

    pdf_vec = []
    for pdf in lha_npdf:
        pdf_vec.append(pdf.xfxQ2(pdfid, pdfx, pdfscale))
    
    #auto xgAll = lha_npdf[0]->xfxQ2(pdfid1, pdfx1, pdfscale);
    xgErr = lha_npdf_set.uncertainty(pdf_vec)

    pdf2_c = xgErr.central / pdfx
    pdf2_up = (xgErr.central + xgErr.errplus) / pdfx
    pdf2_down = (xgErr.central - xgErr.errminus) / pdfx
    #print(pdfx,lha_npdf[0].xfxQ2(pdfid, pdfx, pdfscale))
    pdf_weight_c = 1.0
    pdf_weight_up = 1.0
    pdf_weight_down = 1.0;
    
    if pdfpdf>0.000001 and pdf2_c>0.000001:
        pdf_weight_c =  pdf2_c / pdfpdf;
        pdf_weight_up = pdf2_up / pdfpdf;
        pdf_weight_down = pdf2_down / pdfpdf;
    else:
        a = 0
        #pdf_weight_c = 0;
        #pdf_weight_up = 0;
        #pdf_weight_down = 0;
    
    result_w.append(pdf_weight_c)
    result_w.append(pdf_weight_up)
    result_w.append(pdf_weight_down)
    

In [62]:
# Import the Pythia module.
import pythia8

N_sets = len(lha_npdf_vec)
N_bin_hist= 1 + 3*N_sets


def second(seed, nEvents_tred):
    
    
    hist_Nch = ROOT.TH1D("hist_Nch","hist_Nch",100,0,100)
    hist_xfx = ROOT.TH2D("hist_xfx","hist_xfx",100,0,1,100,0,3)
    hist_part = ROOT.TH3D("hist_part","hist_part",20,0,10,N_bin_hist,0,N_bin_hist,4,0,4)
    

    pythia = pythia8.Pythia()
    pythia.readString(f"Beams:eCM = {Ecm}")
    for iopt in opts:
        pythia.readString(options[iopt])    
    pythia.readString("Random:setSeed = on")
    pythia.readString(f"Random:seed = {seed}")
    pythia.init()
        # Begin event loop. Generate event. Skip if error. List first one.
    for iEvent in range(0, int(nEvents_tred)):
        #print(iEvent, seed)
        if not pythia.next(): continue
        
        info = pythia.infoPython()
        
        weight_pythia = info.weight() 
        
        pdfid1 = info.id1pdf()
        pdfid2 = info.id2pdf()
        pdfx1 = info.x1pdf()
        pdfx2 = info.x2pdf()
        pdfscale = info.Q2Ren()
        pdfpdf1 = info.pdf1()
        pdfpdf2 = info.pdf2()
        xfxQ2 = lha_npdf_vec[0][0].xfxQ2(pdfid1, pdfx1, pdfscale)
        hist_xfx.Fill(pdfx1,xfxQ2,weight_pythia)       
        
        pdfpdf1 = def_npdf[0].xfxQ2(pdfid1, pdfx1, pdfscale) / pdfx1
        pdfpdf2 = def_npdf[0].xfxQ2(pdfid2, pdfx2, pdfscale) / pdfx2
        
        result_projec = [1.0]
        result_target = [1.0]
        for i in range(len(lha_npdf_vec)):
            weights_PDF(result_projec,lha_npdf_vec[i],lha_npdf_set_vec[i],pdfid1,pdfx1,pdfscale,pdfpdf1)
            weights_PDF(result_target,lha_npdf_vec[i],lha_npdf_set_vec[i],pdfid2,pdfx2,pdfscale,pdfpdf2)
        result = result_projec
        for i in range(len(result_projec)):
            result[i]=result_projec[i]*result_target[i]
        #print(result,result_projec,result_target)
        
        # Find number of all final charged particles and fill histogram.
        nCharged = 0
        for pythia_event in pythia.event:  
            if abs(pythia_event.eta()) < 0.5:
                for i in range(N_bin_hist):
                    for iid in range(N_part):
                        if pythia_event.id() == ids[iid]:
                            hist_part.Fill(pythia_event.pT(), i, iid, weight_pythia*result[i])
                        
                
            if pythia_event.isFinal() and pythia_event.isCharged():
                nCharged += 1
        hist_Nch.Fill(nCharged,weight_pythia)
    
        
# End of event loop. Statistics. Histogram. Done.
    pythia.stat()
    return [hist_Nch, hist_xfx, hist_part]

In [63]:
#second(156,5000)

import multiprocess as mp

Ntr = nThread
Nev = nEvents
seed0 = 15156
seed_iter = 43324
pool = mp.Pool(Ntr)
hist_array = pool.starmap(second, [(seed0+i*seed_iter, Nev) for i in range(Ntr)])
    
pool.close()   



 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.308      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 16 Nov 2022  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 05 Dec 2022 at 16:13:52    |  | 
 |  |                                                                              |  | 
 |  |   Program docu

In [64]:
hist_Nch = hist_array[0][0] 
for i in range(1, Ntr):
    hist_Nch.Add(hist_array[i][0])
hist_xfx = hist_array[0][1]
for i in range(1,Ntr):
    hist_xfx.Add(hist_array[i][1]) 
hist_pi0 = hist_array[0][2]
for i in range(1, Ntr):
    hist_pi0.Add(hist_array[i][2])

In [65]:
hists_to_write = []
for ipart in range(N_part,0,-1):
    
    c2 = ROOT.TCanvas(f"c{ipart}",f"c{ipart}",720,720)
    c2.Divide(2,2)
    print(ipart)
#print(len(hist_array),N_bin_hist,len(nPSFs),len(hist_array[0]),hist_pi0.GetNbinsY())
    part_type = ipart
    for i in range(N_sets):
        c2.cd(i+1)
        hist_referen = hist_pi0.ProjectionX("hist_pi0_proj_ref",1,1,part_type,part_type)
        hist_uppredg = hist_pi0.ProjectionX(f"hist_upperedge{i}_{ids[ipart-1]}",3+i*3,3+i*3,part_type,part_type)
        hist_uppredg.Divide(hist_referen)
        hist_uppredg.SetLineColor(6)
        hist_uppredg.Draw()
        hist_central = hist_pi0.ProjectionX(f"hist_central{i}_{ids[ipart-1]}",2+i*3,2+i*3,part_type,part_type)
        hist_central.Divide(hist_referen)
        hist_central.SetLineColor(4)
        hist_central.Draw("same")
        hist_lowedge = hist_pi0.ProjectionX(f"hist_lowedge{i}_{ids[ipart-1]}",4+i*3,4+i*3,part_type,part_type)
        hist_lowedge.Divide(hist_referen)
        hist_lowedge.SetLineColor(2)
        hist_lowedge.Draw("same")
        hists_to_write.append(hist_central)
        hists_to_write.append(hist_lowedge)
        hists_to_write.append(hist_uppredg)
    
    c2.Print(f"NICA/NICA{ids[ipart-1]}.pdf")
    c2.Print(f"NICA/NICA{ids[ipart-1]}.png")
    
c2.Draw()
%jsroot on


4
3
2
1


Info in <TCanvas::Print>: pdf file NICA/NICA-411.pdf has been created
Info in <TCanvas::Print>: png file NICA/NICA-411.png has been created
Info in <TCanvas::Print>: pdf file NICA/NICA411.pdf has been created
Info in <TCanvas::Print>: png file NICA/NICA411.png has been created
Info in <TCanvas::Print>: pdf file NICA/NICA443.pdf has been created
Info in <TCanvas::Print>: png file NICA/NICA443.png has been created
Info in <TCanvas::Print>: pdf file NICA/NICA111.pdf has been created
Info in <TCanvas::Print>: png file NICA/NICA111.png has been created


In [66]:
output_file = ROOT.TFile.Open(f"NICA/dd_outputfile_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.root","RECREATE")
hist_pi0.Write()
for hist in hists_to_write:
    hist.Write()
output_file.Close()

In [70]:
import numpy as np 

Nx = hists_to_write[0].GetNbinsX()+1
pt = [hists_to_write[0].GetBinCenter(i) for i in range(1,Nx)]
data = [[hists_to_write[j].GetBinContent(i) for i in range(1,Nx)] 
               for j in range((N_bin_hist-1)*N_part)]
data.insert(0,pt)
print(data)
np_data = np.array(data)

np.save(f"NICA/data_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.npy", np_data)

data_array = np.load(f"NICA/data_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.npy")
print(data_array)


[[0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.25, 5.75, 6.25, 6.75, 7.25, 7.75, 8.25, 8.75, 9.25, 9.75], [0.0, 0.0, 1.0562596697863087, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.9603185356986939, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.1565429447054707, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.1498059837126402, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0445544651562175, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.2597291392307406, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.567566612965734, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.4063238841970336, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In [74]:
import json

def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

names=['central', 'upper', 'lower']

data_dictionary = {f'{names[j%3]}_{pdf_name[int(j/4)%4]}_{ids[3-int(j/12)]}': [hists_to_write[j].GetBinContent(i) for i in range(1,Nx)] 
               for j in range((N_bin_hist-1)*N_part)}
details = {'pt': pt}
fulldata_dictionary = Merge(details,data_dictionary)

with open(f"NICA/data_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.txt", 'w') as convert_file:
    convert_file.write(json.dumps(fulldata_dictionary))

In [75]:
with open(f"NICA/data_{N_sets}npdf_{int(Nev*nThread)}_at_{int(Ecm)}.txt") as f:
    data = f.read()
    
dictionary = json.loads(data)

print("Data type after reconstruction : ", type(dictionary))
print(dictionary['pt'],dictionary['central_TUJU21_111'])

Data type after reconstruction :  <class 'dict'>
[0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.25, 5.75, 6.25, 6.75, 7.25, 7.75, 8.25, 8.75, 9.25, 9.75] [1.260915302134659, 1.281126140463864, 1.294281080395318, 1.3392257843199147, 1.3811287842889672, 1.6285924032747128, 0.8172523572180597, 0.6933122106756127, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
c0 = ROOT.TCanvas("c0","c0",720,720)

part_type = 2

c0.Divide(2,2)
c0.cd(1)
hist_pi0.Draw()
c0.cd(2)
hist_xfx.Draw()
c0.cd(3)
hist_pi0_proj = hist_pi0.ProjectionX("hist_pi0_proj",2,2,part_type,part_type)
hist_pi0_proj.Draw()
c0.cd(4)
hist_pi0_proj_ref = hist_pi0.ProjectionX("hist_pi0_proj_ref",1,1,part_type,part_type)
R = hist_pi0_proj/hist_pi0_proj_ref
R.Draw("P")
c0.Draw()
%jsroot on
